In [ ]:
import pandas as pd
from datetime import datetime

########### Start of dataPreparation Function #########################
def dataPreparation(dataFrame):
    
    ######### "Airline Column" #############
    dummies = pd.get_dummies(dataFrame.Airline)
    dataFrame[dummies.columns] = dummies
    dataFrame = dataFrame.drop("Airline", axis = 1)
    ########################################
    
    ######## "Source and Destination Column" ########
    dummies = pd.get_dummies(dataFrame.Source)
    columns = []
    for colName in dummies.columns:
         columns.append(colName + "_Source")
    dataFrame[columns] = dummies
    dummies = pd.get_dummies(dataFrame.Destination)
    columns = []
    for colName in dummies.columns:
         columns.append(colName + "_Destination")
    dataFrame[columns] = dummies
    dataFrame = dataFrame.drop(["Source", "Destination"], axis = 1)
    #################################################
    
    ########## "Date_of_Journey Column"##############
    def compare_dates(date):

        date_format = '%d/%m/%Y'
        current_date = datetime.strptime(date, date_format)  
        today = datetime.strptime('01/01/2019', date_format)
        diff = current_date - today
        
        return diff.days

    dataFrame['days_since'] = dataFrame['Date_of_Journey'].apply(compare_dates)
    #################################################
    
    ############### "Route Column" ##################
    dataFrame = dataFrame.drop("Route", axis = 1)
    #################################################
    
    ############## "Duration Column"#################
    def convert_to_minutes(value):
        time = value.split(' ')
        if len(time) == 2:
            return int(time[0][:-1]) * 60 + int(time[1][:-1])
        elif len(time) == 1:
            if time[0][-1] == 'h':
                return int(time[0][:-1]) * 60
            elif time[0][-1] =='m':
                return int(time[0][:-1])
        else:
            return 0
    dataFrame['Duration'] = dataFrame['Duration'].apply(convert_to_minutes)
    #################################################
    
    ############# "Total_Stops" ######################
    def stops_convert_to_numeric(value):
        if str(value)[0] == 'n':
            return 0
        else:
            return int(str(value)[0])
    dataFrame['Total_Stops'] = dataFrame['Total_Stops'].apply(stops_convert_to_numeric)
    ##################################################
    
    ############# "Additional_Info Column" ###############
    def info_convert_to_numeric(value):
        class1 = set(['No info', 'In-flight meal not included', 'No check-in baggage included', 'No Info'])
        class2 = set(['1 Short layover', '1 Long layover', '2 Long layover', 'Change airports', 'Red-eye flight'])
        class3 = set(['Business class'])
        if str(value) in class1:
            return 0
        elif str(value) in class2:
            return 5
        elif str(value) in class3:
            return 20

    dataFrame['Additional_Info'] = dataFrame['Additional_Info'].apply(info_convert_to_numeric)
    ######################################################
    
    ############### "Dep_Time and Arrival_Time Column" #########
    def time_to_numeric(value):
        time = str(value).split(':')
        if int(time[0])  < 7 and int(time[0]) > 19 :
            return 1
        elif int(time[0]) > 15 and int(time[0]) < 19:
            return 4
        elif int(time[0]) > 7 and int(time[0]) < 10:
            return 4
        else:
            return 1

    dataFrame['Dep_Time'] = dataFrame['Dep_Time'].apply(time_to_numeric)
    dataFrame['Arrival_Time'] = dataFrame['Arrival_Time'].apply(time_to_numeric)

    ############################################################
    
    ############### "Adding Column Day_of_Week" ##################
    def week_to_numeric(value):
        class1 = set(["Friday", "Saturday", "Sunday", "Monday"])
        class2 = set(["Tuesday", "Wednesday", "Thursday"])

        if str(value) in class1:
            return 2
        elif str(value) in class2:
            return 0

    dataFrame['Date_of_Journey'] = pd.to_datetime(dataFrame['Date_of_Journey'])
    dataFrame['Day_of_Week'] = dataFrame['Date_of_Journey'].dt.day_name()
    dataFrame['Day_of_Week'] = dataFrame['Day_of_Week'].apply(week_to_numeric)
    dataFrame = dataFrame.drop('Date_of_Journey', axis = 1)
    ##############################################################
    
    ######### splitting dataset into labels and data #############
    labels = dataFrame['Price']
    dataFrame = dataFrame.drop('Price', axis = 1)
    data = dataFrame
    ##############################################################
    
    return data, labels

########## End of dataPreparation Function ############